# Dask DataFrame

Материалы: 
* Макрушин С.В. Лекция 13: Dask DataFrame
* https://docs.dask.org/en/latest/dataframe.html
* Jesse C. Daniel. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Считать данные из файлов в каталоге `accounts`. Содержат ли какие-либо из столбцов пропуски?

In [1]:
import dask.dataframe as dd

In [5]:
accounts = dd.read_csv("accounts/accounts.*.csv",
                       dtype={'amount': 'float64'})
accounts

,id,names,amount
npartitions=3,,,
,int64,object,float64
,...,...,...
,...,...,...
,...,...,...


In [6]:
accounts.isna().sum().compute()

id        0
names     0
amount    3
dtype: int64

2. Подсчитать количество раз, которое то или иное имя встретилось в выборке. Вывести самое часто встречающееся имя.

In [8]:
accounts.groupby('names').size().compute()

names
Alice       185892
Bob         157065
Charlie     109236
Dan          73293
Edith        89991
Frank        99984
George      183249
Hannah      108632
Ingrid      132067
Jerry        96378
Kevin        84784
Laura        97216
Michael     148372
Norbert     188147
Oliver      121907
Patricia     62881
Quinn       118071
Ray         108610
Sarah       104781
Tim         162432
Ursula      108745
Victor      102656
Wendy       110657
Xavier       94445
Yvonne       61462
Zelda        89047
dtype: int64

In [ ]:
accounts['names']

In [14]:
%%time
accounts.groupby('names')['amount'].sum().compute()
accounts.groupby('names')['amount'].mean().compute()

Wall time: 1.34 s


names
Alice        651.642077
Bob          857.655499
Charlie     1745.577526
Dan          920.095002
Edith       1048.074585
Frank       1329.394553
George       935.259292
Hannah      1234.223967
Ingrid      1229.990035
Jerry        513.231526
Kevin       1089.550393
Laura       1985.839851
Michael     2642.594553
Norbert      754.775107
Oliver       400.505664
Patricia     935.283520
Quinn       1232.199880
Ray          505.220709
Sarah        798.997337
Tim         1019.603656
Ursula       339.521881
Victor       756.277665
Wendy        784.367351
Xavier      1787.366944
Yvonne       405.667128
Zelda        700.779296
Name: amount, dtype: float64

In [11]:
import dask

In [15]:
%%time

dask.compute(
    accounts.groupby('names')['amount'].sum().compute(),
    accounts.groupby('names')['amount'].mean().compute()
)

Wall time: 1.48 s


(names
 Alice       121135049.0
 Bob         134707661.0
 Charlie     190678161.0
 Dan          67436523.0
 Edith        94317280.0
 Frank       132918185.0
 George      171385330.0
 Hannah      134076218.0
 Ingrid      162441094.0
 Jerry        49464228.0
 Kevin        92375351.0
 Laura       193055407.0
 Michael     392087039.0
 Norbert     142008672.0
 Oliver       48824444.0
 Patricia     58811563.0
 Quinn       145487072.0
 Ray          54871516.0
 Sarah        83719740.0
 Tim         165616261.0
 Ursula       36921307.0
 Victor       77636440.0
 Wendy        86795738.0
 Xavier      168807871.0
 Yvonne       24933113.0
 Zelda        62402294.0
 Name: amount, dtype: float64,
 names
 Alice        651.642077
 Bob          857.655499
 Charlie     1745.577526
 Dan          920.095002
 Edith       1048.074585
 Frank       1329.394553
 George       935.259292
 Hannah      1234.223967
 Ingrid      1229.990035
 Jerry        513.231526
 Kevin       1089.550393
 Laura       1985.839851
 Mich

3. Создать новую колонку, которая является результатом от деления значения `amount` нацело на 100, если `amount` > 100, и нулём в противном случае.

In [16]:
def f(x):
    if x > 100:
        return x // 100
    return 0

accounts['new_column'] = accounts['amount'].map(f)
accounts.head()

,id,names,amount,new_column
0,39,Victor,205.0,2.0
1,289,Hannah,87.0,0.0
2,234,Victor,1820.0,18.0
3,155,Ray,-600.0,0.0
4,266,Ray,572.0,5.0


## Лабораторная работа 13

__При решении данных задач не подразумевается использования других коллекций, кроме `dask.DataFrame`, если в задании явно не указано обратное.__

In [1]:
import dask.dataframe as dd
import pandas as pd
import dask.bag as db
import json
import dask

1. В архиве `recipes_full.zip` находятся файлы, содержащие информацию об рецептах блюд. Загрузите данные из файлов этого архива в виде `dd.DataFrame` с названием `recipes`. Укажите, что в столбце `submitted` содержатся даты.

2. Выведите метаинформацию о таблице: `npartitions` и типы столбцов.

3. Выведите на экран 5 первых строк таблицы. Выведите на экран 5 последних строк таблицы. В случае сообщения об ошибки объясните причину и исправьте ошибку.

4. Посчитайте, сколько строк содержит каждый из блоков (partitions).

5. Найдите максимум в столбце `n_steps`. Визуализируйте граф вычислений для этой задачи. Прокомментируйте логику работы `dask` при решении данной задачи.

6. Посчитайте количество рецептов с группировкой по месяцам добавления отзыва в базу.

7. Считайте файлы из архива `reviews_full.zip` (__ЛР12__) в виде `dask.bag`. Пользуясь результатом лабораторной работы 12, рассчитайте среднее значение оценок отзывов с группировкой по месяцам. После завершения всех вычислений преобразуйте результат к `pd.Series`.

8. Пользуясь результатами решения задач 6 и 7, создайте `pd.DataFrame`, содержащий два столбца: `mean_rating`, `recipes_count`